In [ ]:
# https://scikit-learn.org/stable/modules/multiclass.html

In [1]:
# Import libraries
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import requests
import matplotlib.pyplot as plt
import json
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# # Read csv and create dataframe
# from google.colab import files
# uploaded = files.upload()

In [115]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import io
salaries_df = pd.read_csv('salaries_cleaned.csv')

In [4]:
layoffs_df = pd.read_csv('layoffs_cleaned.csv')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
salaries_df.head()

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education
0,0,2017-06-07 11:33:00,2017-06-07 00:00:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown
1,1,2017-06-10 17:11:00,2017-06-10 00:00:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown
2,2,2017-06-11 14:53:00,2017-06-11 00:00:00,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown
3,3,2017-06-17 00:23:00,2017-06-17 00:00:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown
4,4,2017-06-20 10:58:00,2017-06-20 00:00:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown


In [6]:
layoffs_df

,Unnamed: 0,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country
0,0,N26,Berlin,Finance,71,4.0,2023-04-28,Series E,United States
1,1,Providoor,Melbourne,Food,0,100.0,2023-04-28,Unknown,Australia
2,2,Dropbox,SF Bay Area,Other,500,16.0,2023-04-27,Post-IPO,United States
3,3,Vroom,New York City,Transportation,120,11.0,2023-04-27,Post-IPO,United States
4,4,Greenhouse,New York City,Recruiting,100,12.0,2023-04-27,Private Equity,United States
...,...,...,...,...,...,...,...,...,...
2540,2540,Panda Squad,SF Bay Area,Consumer,6,75.0,2020-03-13,Seed,United States
2541,2541,Tamara Mellon,Los Angeles,Retail,20,40.0,2020-03-12,Series C,United States
2542,2542,EasyPost,Salt Lake City,Logistics,75,NaN,2020-03-11,Series A,United States
2543,2543,Blackbaud,Charleston,Other,500,14.0,NaN,Post-IPO,United States


In [7]:
layoffs_df_update=layoffs_df[['company', 'total_laid_off', 'percentage_laid_off']]
layoffs_df_update

,company,total_laid_off,percentage_laid_off
0,N26,71,4.0
1,Providoor,0,100.0
2,Dropbox,500,16.0
3,Vroom,120,11.0
4,Greenhouse,100,12.0
...,...,...,...
2540,Panda Squad,6,75.0
2541,Tamara Mellon,20,40.0
2542,EasyPost,75,NaN
2543,Blackbaud,500,14.0


In [8]:
layoff_list=layoffs_df_update['company'].unique()
layoff_list

array(['N26', 'Providoor', 'Dropbox', ..., 'Tamara Mellon', 'Blackbaud',
       'Open'], dtype=object)

In [9]:
salaries_df['layoff_happened']=0

In [10]:
salaries_df

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education,layoff_happened
0,0,2017-06-07 11:33:00,2017-06-07 00:00:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown,0
1,1,2017-06-10 17:11:00,2017-06-10 00:00:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,0
2,2,2017-06-11 14:53:00,2017-06-11 00:00:00,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown,0
3,3,2017-06-17 00:23:00,2017-06-17 00:00:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown,0
4,4,2017-06-20 10:58:00,2017-06-20 00:00:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,62637,2018-09-09 11:52:00,2018-09-09 00:00:00,Google,T4,Software Engineer,327000,"Seattle, WA",47.603832,-122.330062,10.0,1.0,155000,150000,22000,Unknown,Unknown,Unknown,0
62638,62638,2018-09-13 08:23:00,2018-09-13 00:00:00,Microsoft,62,Software Engineer,237000,"Redmond, WA",47.669414,-122.123877,2.0,2.0,146900,73200,16000,Unknown,Unknown,Unknown,0
62639,62639,2018-09-13 14:35:00,2018-09-13 00:00:00,MSFT,63,Software Engineer,220000,"Seattle, WA",47.603832,-122.330062,14.0,12.0,157000,25000,20000,Unknown,Unknown,Unknown,0
62640,62640,2018-09-16 16:10:00,2018-09-16 00:00:00,Salesforce,Lead MTS,Software Engineer,280000,"San Francisco, CA",37.779026,-122.419906,8.0,4.0,194688,57000,29000,Unknown,Unknown,Unknown,0


In [11]:
for x in range(len(salaries_df['company'])):
    if salaries_df['company'][x] in layoff_list:
        salaries_df['layoff_happened'][x] = 1
      


In [12]:
salaries_df['layoff_happened'].value_counts()

1    40410
0    22232
Name: layoff_happened, dtype: int64

In [13]:
salaries_df.columns

Index(['Unnamed: 0', 'timestamp', 'date', 'company', 'level', 'title',
       'totalyearlycompensation', 'location', 'latitude', 'longitude',
       'yearsofexperience', 'yearsatcompany', 'basesalary', 'stockgrantvalue',
       'bonus', 'gender', 'Race', 'Education', 'layoff_happened'],
      dtype='object')

,title
Human Resources,364
Recruiter,451
Sales,461
Mechanical Engineer,490
Marketing,710
Business Analyst,885
Management Consultant,976
Solution Architect,1157
Technical Program Manager,1381
Product Designer,1516


In [14]:
salaries_df['company'].value_counts()

Amazon                      8126
Microsoft                   5216
Google                      4330
Facebook                    2990
Apple                       2028
                            ... 
Samsung research America       1
Bny Mellon                     1
yelp                           1
Bloomberg lp                   1
tableau software               1
Name: company, Length: 1631, dtype: int64

In [15]:
val=salaries_df['company'].value_counts()
print(val[val>25])

Amazon           8126
Microsoft        5216
Google           4330
Facebook         2990
Apple            2028
                 ... 
TSMC               26
Huawei             26
Gojek Tech         26
Fortinet           26
Credit Suisse      26
Name: company, Length: 259, dtype: int64


In [16]:
#companies over 25 jobs
comapanies_to_replace=[]
company=salaries_df['company'].value_counts()
company=pd.DataFrame(company)
company.head(20)

,company
Amazon,8126
Microsoft,5216
Google,4330
Facebook,2990
Apple,2028
Oracle,1128
Salesforce,1056
Intel,949
Cisco,907
IBM,907


In [17]:
comapanies_to_replace.append(company[company.company<15].index)

AttributeError: 'list' object has no attribute 'value_counts'

In [18]:
#Replaced with other for all companies besides the top 20
for x in comapanies_to_replace:
    salaries_df['company'] = salaries_df['company'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['company'].value_counts()

Amazon         8126
Other          6259
Microsoft      5216
Google         4330
Facebook       2990
               ... 
LendingClub      15
Bosch            15
Cadence          15
Freshworks       15
BAE Systems      15
Name: company, Length: 393, dtype: int64

In [19]:
year_df=salaries_df['date'].str.split('-', expand=True)

In [20]:
salaries_df['date']=year_df[0]

In [21]:
salaries_df['month']=year_df[1]

In [22]:
salaries_df

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education,layoff_happened,month
0,0,2017-06-07 11:33:00,2017,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown,1,06
1,1,2017-06-10 17:11:00,2017,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,1,06
2,2,2017-06-11 14:53:00,2017,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown,1,06
3,3,2017-06-17 00:23:00,2017,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown,1,06
4,4,2017-06-20 10:58:00,2017,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,1,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,62637,2018-09-09 11:52:00,2018,Google,T4,Software Engineer,327000,"Seattle, WA",47.603832,-122.330062,10.0,1.0,155000,150000,22000,Unknown,Unknown,Unknown,1,09
62638,62638,2018-09-13 08:23:00,2018,Microsoft,62,Software Engineer,237000,"Redmond, WA",47.669414,-122.123877,2.0,2.0,146900,73200,16000,Unknown,Unknown,Unknown,1,09
62639,62639,2018-09-13 14:35:00,2018,Other,63,Software Engineer,220000,"Seattle, WA",47.603832,-122.330062,14.0,12.0,157000,25000,20000,Unknown,Unknown,Unknown,0,09
62640,62640,2018-09-16 16:10:00,2018,Salesforce,Lead MTS,Software Engineer,280000,"San Francisco, CA",37.779026,-122.419906,8.0,4.0,194688,57000,29000,Unknown,Unknown,Unknown,1,09


In [23]:
salaries_df['level'].value_counts()

L4                              5014
L5                              4871
L3                              3337
L6                              2871
Senior Software Engineer        1443
                                ... 
Gr 7                               1
Senior BA                          1
Consulting Analyst                 1
Senior Technical Director II       1
Bioinformatics Scientist II        1
Name: level, Length: 2916, dtype: int64

In [24]:
#companies over 25 jobs
levels_to_replace=[]
levels=salaries_df['level'].value_counts()
levels=pd.DataFrame(levels)
levels.head(20)

,level
L4,5014
L5,4871
L3,3337
L6,2871
Senior Software Engineer,1443
L2,1163
Senior,1079
L7,919
L1,765
62,764


In [25]:
levels_to_replace.append(levels[levels.level<15].index)

In [26]:
for x in levels_to_replace:
    salaries_df['level'] = salaries_df['level'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['level'].value_counts()

Other                            5668
L4                               5014
L5                               4871
L3                               3337
L6                               2871
                                 ... 
New grad                           15
Engineer I                         15
Only one level across Netflix      15
Grade 14                           15
PR3                                15
Name: level, Length: 326, dtype: int64

In [27]:
salaries_df

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education,layoff_happened,month
0,0,2017-06-07 11:33:00,2017,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown,1,06
1,1,2017-06-10 17:11:00,2017,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,1,06
2,2,2017-06-11 14:53:00,2017,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown,1,06
3,3,2017-06-17 00:23:00,2017,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown,1,06
4,4,2017-06-20 10:58:00,2017,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown,1,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,62637,2018-09-09 11:52:00,2018,Google,T4,Software Engineer,327000,"Seattle, WA",47.603832,-122.330062,10.0,1.0,155000,150000,22000,Unknown,Unknown,Unknown,1,09
62638,62638,2018-09-13 08:23:00,2018,Microsoft,62,Software Engineer,237000,"Redmond, WA",47.669414,-122.123877,2.0,2.0,146900,73200,16000,Unknown,Unknown,Unknown,1,09
62639,62639,2018-09-13 14:35:00,2018,Other,63,Software Engineer,220000,"Seattle, WA",47.603832,-122.330062,14.0,12.0,157000,25000,20000,Unknown,Unknown,Unknown,0,09
62640,62640,2018-09-16 16:10:00,2018,Salesforce,Lead MTS,Software Engineer,280000,"San Francisco, CA",37.779026,-122.419906,8.0,4.0,194688,57000,29000,Unknown,Unknown,Unknown,1,09


In [28]:
reduced_df = salaries_df[[ 'company', 'date', 'title', 'totalyearlycompensation', 'gender', 'level',
       'yearsofexperience', 'yearsatcompany', 'latitude', 'longitude', 'month', 'bonus', 'stockgrantvalue', 'layoff_happened']]

In [29]:
reduced_df


,company,date,title,totalyearlycompensation,gender,level,yearsofexperience,yearsatcompany,latitude,longitude,month,bonus,stockgrantvalue,layoff_happened
0,Oracle,2017,Product Manager,127000,Unknown,L3,1.5,1.5,37.486324,-122.232523,06,10000,20000,1
1,eBay,2017,Software Engineer,100000,Unknown,SE 2,5.0,3.0,37.779026,-122.419906,06,0,0,1
2,Amazon,2017,Product Manager,310000,Unknown,L7,8.0,0.0,47.603832,-122.330062,06,0,0,1
3,Apple,2017,Software Engineering Manager,372000,Unknown,M1,7.0,5.0,37.368830,-122.036349,06,35000,180000,1
4,Microsoft,2017,Software Engineer,157000,Unknown,60,5.0,3.0,37.389389,-122.083210,06,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,Google,2018,Software Engineer,327000,Unknown,T4,10.0,1.0,47.603832,-122.330062,09,22000,150000,1
62638,Microsoft,2018,Software Engineer,237000,Unknown,62,2.0,2.0,47.669414,-122.123877,09,16000,73200,1
62639,Other,2018,Software Engineer,220000,Unknown,63,14.0,12.0,47.603832,-122.330062,09,20000,25000,0
62640,Salesforce,2018,Software Engineer,280000,Unknown,Lead MTS,8.0,4.0,37.779026,-122.419906,09,29000,57000,1


In [30]:
dummies_df = pd.get_dummies(reduced_df)
dummies_df.head()

,totalyearlycompensation,yearsofexperience,yearsatcompany,latitude,longitude,bonus,stockgrantvalue,layoff_happened,company_3M,company_ADP,...,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,127000,1.5,1.5,37.486324,-122.232523,10000,20000,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,100000,5.0,3.0,37.779026,-122.419906,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,310000,8.0,0.0,47.603832,-122.330062,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,372000,7.0,5.0,37.368830,-122.036349,35000,180000,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,157000,5.0,3.0,37.389389,-122.083210,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(columns=['layoff_happened']).values
y = dummies_df['layoff_happened'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# # Split our preprocessed data into our features and target arrays
# X = reduced_df.drop(columns=["totalyearlycompensation"]).values
# y = reduced_df["totalyearlycompensation"].values

# # Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [32]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X.shape

(62642, 1648)

In [33]:
#linear regression
# Import required libraries
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.linear_model import LinearRegression

In [ ]:
#model = LinearRegression()

In [ ]:
#y = dummies_df["totalyearlycompensation"].values


In [34]:
#random forest
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [28]:
#https://stackoverflow.com/questions/52648383/how-to-get-coefficients-and-feature-importances-from-multioutputregressor
#rf_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=500, random_state=78, max_depth=20))

In [35]:
rf_model = RandomForestClassifier(n_estimators=500, random_state=78, max_depth=20, n_jobs=-1, verbose=3)
#rf_model = RandomForestRegressor(n_estimators=500, random_state=7, max_depth=10)

In [36]:
rf_model = rf_model.fit(X_train_scaled, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 2 of 500building tree 3 of 500
building tree 1 of 500
building tree 4 of 500

building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.7s


building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   24.9s


building tree 121 of 500
building tree 122 of 500
building tree 123 of 500
building tree 124 of 500
building tree 125 of 500
building tree 126 of 500
building tree 127 of 500
building tree 128 of 500
building tree 129 of 500
building tree 130 of 500
building tree 131 of 500
building tree 132 of 500
building tree 133 of 500
building tree 134 of 500
building tree 135 of 500
building tree 136 of 500
building tree 137 of 500
building tree 138 of 500
building tree 139 of 500
building tree 140 of 500building tree 141 of 500

building tree 142 of 500
building tree 143 of 500
building tree 144 of 500
building tree 145 of 500
building tree 146 of 500
building tree 147 of 500
building tree 148 of 500
building tree 149 of 500
building tree 150 of 500
building tree 151 of 500
building tree 152 of 500
building tree 153 of 500
building tree 154 of 500
building tree 155 of 500
building tree 156 of 500
building tree 157 of 500
building tree 158 of 500
building tree 159 of 500
building tree 160 of 500


[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   60.0s


building tree 282 of 500
building tree 283 of 500
building tree 284 of 500
building tree 285 of 500
building tree 286 of 500
building tree 287 of 500
building tree 288 of 500
building tree 289 of 500
building tree 290 of 500
building tree 291 of 500
building tree 292 of 500
building tree 293 of 500
building tree 294 of 500
building tree 295 of 500
building tree 296 of 500
building tree 297 of 500
building tree 298 of 500building tree 299 of 500

building tree 300 of 500
building tree 301 of 500
building tree 302 of 500
building tree 303 of 500
building tree 304 of 500
building tree 305 of 500
building tree 306 of 500
building tree 307 of 500
building tree 308 of 500
building tree 309 of 500
building tree 310 of 500
building tree 311 of 500
building tree 312 of 500
building tree 313 of 500
building tree 314 of 500
building tree 315 of 500
building tree 316 of 500
building tree 317 of 500
building tree 318 of 500
building tree 319 of 500
building tree 320 of 500
building tree 321 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.8min finished


In [38]:
predictions = rf_model.predict(X_test_scaled)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.8s finished


In [40]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8938126556414022

In [190]:
acc_score = rf_model.score(X_test_scaled, y_test)
acc_score
#0.7402413883001011 with bonus
#0.8639637152876211 with stock
#0.6690924280066902 with bonus and stock as categorical data only

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.7s finished


0.8938126556414022

In [41]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [42]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4440,1194
Actual 1,469,9558


Accuracy Score : 0.8938126556414022
Classification Report
              precision    recall  f1-score   support

           0       0.90      0.79      0.84      5634
           1       0.89      0.95      0.92     10027

    accuracy                           0.89     15661
   macro avg       0.90      0.87      0.88     15661
weighted avg       0.89      0.89      0.89     15661



In [43]:
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(cm, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2)

NameError: name 'sns' is not defined

<Figure size 1600x700 with 0 Axes>

In [ ]:
#deep learning

In [ ]:

#https://www.simplilearn.com/tutorials/machine-learning-tutorial/stock-price-prediction-using-machine-learning
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = (len(dummies_df.columns)-1)
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.LSTM(units=80, return_sequences=True, input_shape=(X_train.shape[1],1)))

# Second hidden layer
nn.add(tf.keras.layers.LSTM(units=30, return_sequences=True))

# Output layer
nn.add(tf.keras.layers.Dense(units=1))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 37, 80)            26240     
                                                                 
 lstm_1 (LSTM)               (None, 37, 30)            13320     
                                                                 
 dense (Dense)               (None, 37, 1)             31        
                                                                 
Total params: 39,591
Trainable params: 39,591
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1469/1469 [==============================] - 102s 63ms/step - loss: -3294135.2500 - accuracy: 0.0000e+00
Epoch 2/100
1469/1469 [==============================] - 93s 63ms/step - loss: -3294138.2500 - accuracy: 0.0000e+00
Epoch 3/100
1469/1469 [==============================] - 92s 63ms/step - loss: -3294136.0000 - accuracy: 0.0000e+00
Epoch 4/100
1469/1469 [==============================] - 93s 63ms/step - loss: -3294141.7500 - accuracy: 0.0000e+00
Epoch 5/100
1469/1469 [==============================] - 94s 64ms/step - loss: -3294133.2500 - accuracy: 0.0000e+00
Epoch 6/100
1469/1469 [==============================] - 94s 64ms/step - loss: -3294137.7500 - accuracy: 0.0000e+00
Epoch 7/100
 240/1469 [===>..........................] - ETA: 1:16 - loss: -3308527.0000 - accuracy: 0.0000e+00

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")